# Old metrics benchmark

This notebook shows the improvements in speed of new metrics (#47) against the old ones

In [1]:
%load_ext autoreload
%autoreload 2

# Necessary imports
import os
import sys
import warnings

import numpy as np
import cupy as cp

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

sys.path.insert(0, '../../..')
from seismiqb import Horizon, SeismicGeometry
from seismiqb import plot_image, plot_loss, METRIC_CMAP

from seismiqb.src.metrics import smooth_out

# Load geometry and horizon

In [2]:
cube_path = '/data/seismic_data/seismic_interpretation/CUBE_01_ETP/amplitudes_01_ETP.hdf5'
horizon_path = '/data/seismic_data/seismic_interpretation/CUBE_01_ETP/INPUTS/HORIZONS/RAW/etp_BP12_anon'

geometry = SeismicGeometry(cube_path)
horizon = Horizon(horizon_path, geometry=geometry)

# Horizon metrics

### Load data along horizon

In [3]:
from seismiqb import HorizonMetrics as HMOLD
from seismiqb.src.new_metrics import HorizonMetrics as HMNEW

In [4]:
%%time
hmold = HMOLD(horizon)
data = hmold.data

CPU times: user 3.07 s, sys: 1.07 s, total: 4.14 s
Wall time: 4.14 s


In [5]:
%%time
hmnew = HMNEW(horizon)
data = hmnew.data

CPU times: user 3.07 s, sys: 1.06 s, total: 4.12 s
Wall time: 4.12 s


### Local corrs

In [6]:
%time old_lc = hmold.evaluate('local_corrs', kernel_size=3, reduce_func='nanmean')
%time new_lc_cpu = hmnew.evaluate('local_corrs', kernel_size=3, agg='nanmean')
%time new_lc_gpu = hmnew.evaluate('local_corrs', kernel_size=3, agg='nanmean', device='gpu')

np.nanmean(np.abs(old_lc - new_lc_gpu)), np.nanstd(np.abs(old_lc - new_lc_gpu))

CPU times: user 6min 34s, sys: 1.84 s, total: 6min 36s
Wall time: 8.27 s
CPU times: user 1.84 s, sys: 1.03 s, total: 2.87 s
Wall time: 8.64 s
CPU times: user 493 ms, sys: 419 ms, total: 913 ms
Wall time: 1.21 s


(4.4382989036859414e-08, 3.7101049080114866e-08)

### Support corrs

In [7]:
supports = [[item, item] for item in range(555, 955, 4)]
print(len(supports))

%time old_corrs = hmold.evaluate('support_corrs', supports=supports, plot=False)
%time new_corrs_cpu = hmnew.evaluate('support_corrs', supports=supports, plot=False)
%time new_corrs_gpu = hmnew.evaluate('support_corrs', supports=supports, device='gpu', plot=False)

np.nanmean(np.abs(old_corrs - new_corrs_gpu)), np.nanstd(np.abs(old_corrs - new_corrs_gpu))

100
CPU times: user 27.1 s, sys: 14.3 s, total: 41.4 s
Wall time: 41.4 s
CPU times: user 14.9 s, sys: 7.04 s, total: 22 s
Wall time: 22 s
CPU times: user 448 ms, sys: 327 ms, total: 775 ms
Wall time: 821 ms


(1.492855259786126e-08, 1.7873892629261548e-08)

### Support crosscorrs

In [8]:
supports = [[item, item] for item in range(555, 955, 4)]
print(len(supports))

%time old_cc = hmold.evaluate('support_crosscorrs', supports=supports, agg='nanmean', plot=False)
%time new_cc_cpu = hmnew.evaluate('support_crosscorrs', supports=supports, agg='nanmean', plot=False)
%time new_cc_gpu = hmnew.evaluate('support_crosscorrs', supports=supports, agg='nanmean', device='gpu', plot=False)

np.nanmean(np.abs(old_cc - new_cc_cpu)), np.nanstd(np.abs(old_cc - new_cc_cpu))

100
CPU times: user 8min 54s, sys: 4min 53s, total: 13min 48s
Wall time: 13min 48s
CPU times: user 4min 45s, sys: 2min 27s, total: 7min 13s
Wall time: 7min 13s
CPU times: user 5.13 s, sys: 3.58 s, total: 8.71 s
Wall time: 8.81 s


(0.014807187488141011, 0.03468628590993994)

### Instantaneous phases

In [9]:
%time old_ip = hmold.evaluate('instantaneous_phase', plot=False)
%time new_ip_cpu = hmnew.evaluate('instantaneous_phase', plot=False)
%time new_ip_gpu = hmnew.evaluate('instantaneous_phase', device='gpu', plot=False)

np.nanmean(np.abs(old_ip - new_ip_cpu)), np.nanstd(np.abs(old_ip - new_ip_cpu))

CPU times: user 11.3 s, sys: 1.3 s, total: 12.6 s
Wall time: 12.6 s
CPU times: user 10.7 s, sys: 989 ms, total: 11.7 s
Wall time: 11.7 s
CPU times: user 389 ms, sys: 155 ms, total: 543 ms
Wall time: 620 ms


(2.0162179065792003, 1.0635306250595937)

# Geometry metrics

### Load histogram matrices

In [10]:
from seismiqb import GeometryMetrics as GMOLD
from seismiqb.src.new_metrics import GeometryMetrics as GMNEW

In [11]:
%%time
gmold = GMOLD(geometry)
data = gmold.data
probs = gmold.probs

CPU times: user 242 ms, sys: 392 ms, total: 634 ms
Wall time: 632 ms


In [12]:
%%time
gmnew = GMNEW(geometry)
data = gmnew.data
probs = gmnew.probs

CPU times: user 251 ms, sys: 384 ms, total: 635 ms
Wall time: 633 ms


### Support metrics

In [13]:
S = [[item, item] for item in range(555, 755, 5)]
print('Number of supports:', len(S))

lst = [item for item in GMNEW.AVAILABLE_METRICS if 'support' in item]

for metric_name in lst:
    old_value = gmold.evaluate(metric_name, supports=S)
    new_value_cpu = gmnew.evaluate(metric_name, supports=S, device='cpu')
    new_value_gpu = gmnew.evaluate(metric_name, supports=S, device='gpu')
    flag = np.isclose(old_value, new_value_cpu, equal_nan=True).all()
    print(f'\n{metric_name:20} ::: {flag}')
    if not flag:
        print(f'    Difference: {np.nanmean(np.abs(old_value - new_value_cpu))}')

    %timeit -n 2 -r 2 old_value = gmold.evaluate(metric_name, supports=S)
    %timeit -n 2 -r 2 new_value_cpu = gmnew.evaluate(metric_name, supports=S, device='cpu', amortize=True)
    %timeit -n 2 -r 2 new_value_gpu = gmnew.evaluate(metric_name, supports=S, device='gpu')

Number of supports: 40

support_corrs        ::: True
17.6 s ± 2.31 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
14.9 s ± 5.94 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
581 ms ± 13.8 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

support_btch         ::: True
24.7 s ± 34.8 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
22.3 s ± 11.9 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
696 ms ± 25.6 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

support_kl           ::: True
49.1 s ± 10.4 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
46.8 s ± 4.26 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
1.09 s ± 20.5 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

support_js           ::: True
1min 51s ± 13.4 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
1min 49s ± 16.2 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
2.16 s ± 18.7 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

support_hellinger   

### Local metrics

In [14]:
KS = 5

lst = [item for item in GMNEW.AVAILABLE_METRICS if 'local' in item]

for metric_name in lst:

    old_value = gmold.evaluate(metric_name, kernel_size=KS)
    new_value_cpu = gmnew.evaluate(metric_name, kernel_size=KS, device='cpu')
    new_value_gpu = gmnew.evaluate(metric_name, kernel_size=KS, device='gpu')
    flag = np.isclose(old_value, new_value_cpu, equal_nan=True).all()
    print(f'\n{metric_name:20} ::: {flag}')
    if not flag:
        print(f'    Difference: {np.nanmean(np.abs(old_value - new_value_cpu))}')
    
    %timeit -n 2 -r 2 old_value = gmold.evaluate(metric_name, kernel_size=KS)
    %timeit -n 2 -r 2 new_value_cpu = gmnew.evaluate(metric_name, kernel_size=KS, device='cpu')
    %timeit -n 2 -r 2 new_value_gpu = gmnew.evaluate(metric_name, kernel_size=KS, device='gpu')


local_corrs          ::: True
15.7 s ± 155 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
10 s ± 3.14 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
478 ms ± 7.93 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

local_btch           ::: True
14.1 s ± 75.9 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
14.3 s ± 1.11 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
569 ms ± 25.5 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

local_kl             ::: False
    Difference: 0.001375491291533084
13.9 s ± 30.5 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
26.2 s ± 10.5 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
801 ms ± 9.97 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

local_js             ::: True
53 s ± 129 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
1min 19s ± 9.93 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
1.73 s ± 36.4 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)

local_helling